In [1]:
import nltk
import numpy as np

from sklearn.model_selection import train_test_split

from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, LSTM, InputLayer, Bidirectional, TimeDistributed, Embedding, Activation
from keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [2]:
tagged_sentences = nltk.corpus.treebank.tagged_sents()
 
print(tagged_sentences[0])
print("Tagged sentences: ", len(tagged_sentences))
print("Tagged words:", len(nltk.corpus.treebank.tagged_words()))

[('Pierre', 'NNP'), ('Vinken', 'NNP'), (',', ','), ('61', 'CD'), ('years', 'NNS'), ('old', 'JJ'), (',', ','), ('will', 'MD'), ('join', 'VB'), ('the', 'DT'), ('board', 'NN'), ('as', 'IN'), ('a', 'DT'), ('nonexecutive', 'JJ'), ('director', 'NN'), ('Nov.', 'NNP'), ('29', 'CD'), ('.', '.')]
Tagged sentences:  3914
Tagged words: 100676


In [3]:
sentences, sentence_tags =[], [] 
for tagged_sentence in tagged_sentences:
    sentence, tags = zip(*tagged_sentence)
    sentences.append(np.array(sentence))
    sentence_tags.append(np.array(tags))
 
print(sentences[0])
print(sentence_tags[0])

['Pierre' 'Vinken' ',' '61' 'years' 'old' ',' 'will' 'join' 'the' 'board'
 'as' 'a' 'nonexecutive' 'director' 'Nov.' '29' '.']
['NNP' 'NNP' ',' 'CD' 'NNS' 'JJ' ',' 'MD' 'VB' 'DT' 'NN' 'IN' 'DT' 'JJ'
 'NN' 'NNP' 'CD' '.']


In [4]:
(train_sentences, 
 test_sentences, 
 train_tags, 
 test_tags) = train_test_split(sentences, sentence_tags, test_size=0.2)

In [5]:
words, tags = set([]), set([])
 
for s in train_sentences:
    for w in s:
        words.add(w.lower())
 
for ts in train_tags:
    for t in ts:
        tags.add(t)
 
word2index = {w: i + 2 for i, w in enumerate(list(words))}
word2index['-PAD-'] = 0  # The special value used for padding
word2index['-OOV-'] = 1  # The special value used for OOVs
 
tag2index = {t: i + 1 for i, t in enumerate(list(tags))}
tag2index['-PAD-'] = 0  # The special value used to padding

In [6]:
train_sentences_X, test_sentences_X, train_tags_y, test_tags_y = [], [], [], []
 
for s in train_sentences:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])
 
    train_sentences_X.append(s_int)
 
for s in test_sentences:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])
 
    test_sentences_X.append(s_int)
 
for s in train_tags:
    train_tags_y.append([tag2index[t] for t in s])
 
for s in test_tags:
    test_tags_y.append([tag2index[t] for t in s])
 
print(train_sentences_X[0])
print(test_sentences_X[0])
print(train_tags_y[0])
print(test_tags_y[0])

[1448, 8001, 8141, 3367, 8664, 6308, 5316, 6641, 6491, 8246, 2244, 8661, 355, 974, 3044, 4605, 7642, 5525, 912, 3890, 2892, 8001, 4599, 4869, 4515, 3942, 13, 6624, 6205, 3832, 1386, 3857, 4927, 2892, 8723, 2207, 1265, 6980, 737, 9597, 7306, 6789, 8246, 4593, 6550, 1715, 4927, 4266, 9597, 4850, 7306, 9766, 1959, 7585, 4604, 6912]
[5138, 7963, 4736, 2320, 3963, 924, 9597, 3762, 9985, 5715, 9707, 9844, 2244, 2661, 1, 6912]
[30, 26, 40, 8, 18, 29, 39, 2, 41, 28, 20, 43, 14, 30, 11, 44, 8, 13, 13, 33, 14, 26, 41, 31, 14, 8, 42, 30, 26, 30, 40, 8, 31, 14, 41, 8, 10, 42, 8, 19, 42, 8, 28, 30, 41, 24, 31, 14, 19, 14, 42, 41, 30, 40, 32, 7]
[5, 26, 40, 41, 1, 29, 19, 26, 41, 29, 1, 13, 20, 18, 40, 7]


In [7]:
MAX_LENGTH = len(max(train_sentences_X, key=len))
print(MAX_LENGTH)  # 271

125


In [8]:
train_sentences_X = pad_sequences(train_sentences_X, maxlen=MAX_LENGTH, padding='post')
test_sentences_X = pad_sequences(test_sentences_X, maxlen=MAX_LENGTH, padding='post')
train_tags_y = pad_sequences(train_tags_y, maxlen=MAX_LENGTH, padding='post')
test_tags_y = pad_sequences(test_tags_y, maxlen=MAX_LENGTH, padding='post')
 
print(train_sentences_X[0])
print(test_sentences_X[0])
print(train_tags_y[0])
print(test_tags_y[0])

[1448 8001 8141 3367 8664 6308 5316 6641 6491 8246 2244 8661  355  974
 3044 4605 7642 5525  912 3890 2892 8001 4599 4869 4515 3942   13 6624
 6205 3832 1386 3857 4927 2892 8723 2207 1265 6980  737 9597 7306 6789
 8246 4593 6550 1715 4927 4266 9597 4850 7306 9766 1959 7585 4604 6912
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0]
[5138 7963 4736 2320 3963  924 9597 3762 9985 5715 9707 9844 2244 2661
    1 6912    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0

In [14]:
from keras import backend as K
 
def ignore_class_accuracy(to_ignore=0):
    def ignore_accuracy(y_true, y_pred):
        y_true_class = K.argmax(y_true, axis=-1)
        y_pred_class = K.argmax(y_pred, axis=-1)
 
        ignore_mask = K.cast(K.not_equal(y_pred_class, to_ignore), 'int32')
        matches = K.cast(K.equal(y_true_class, y_pred_class), 'int32') * ignore_mask
        accuracy = K.sum(matches) / K.maximum(K.sum(ignore_mask), 1)
        return accuracy
    return ignore_accuracy

In [15]:
model = Sequential()
model.add(InputLayer(input_shape=(MAX_LENGTH, )))
model.add(Embedding(len(word2index), 128))
model.add(Bidirectional(LSTM(256, return_sequences=True)))
model.add(TimeDistributed(Dense(len(tag2index))))
model.add(Activation('softmax'))
 
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(0.001),
              metrics=['accuracy', ignore_class_accuracy(0)])
 
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 125, 128)          1295744   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 125, 512)          788480    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 125, 47)           24111     
_________________________________________________________________
activation_2 (Activation)    (None, 125, 47)           0         
Total params: 2,108,335
Trainable params: 2,108,335
Non-trainable params: 0
_________________________________________________________________


In [16]:
def to_categorical(sequences, categories):
    cat_sequences = []
    for s in sequences:
        cats = []
        for item in s:
            cats.append(np.zeros(categories))
            cats[-1][item] = 1.0
        cat_sequences.append(cats)
    return np.array(cat_sequences)

In [17]:
cat_train_tags_y = to_categorical(train_tags_y, len(tag2index))
print(cat_train_tags_y[0])

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


In [18]:
model.fit(train_sentences_X, to_categorical(train_tags_y, len(tag2index)), batch_size=128, epochs=40, validation_split=0.2)

C:\Users\LENOVO\anaconda3\envs\tensorflow\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 2504 samples, validate on 627 samples
Epoch 1/40
2504/2504 [==============================] - 63s 25ms/step - loss: 1.5466 - accuracy: 0.7576 - ignore_accuracy: 0.0510 - val_loss: 0.7647 - val_accuracy: 0.8180 - val_ignore_accuracy: 0.1293
Epoch 2/40
2504/2504 [==============================] - 90s 36ms/step - loss: 0.7029 - accuracy: 0.8006 - ignore_accuracy: 0.0916 - val_loss: 0.6826 - val_accuracy: 0.8013 - val_ignore_accuracy: 0.1121
Epoch 3/40
2504/2504 [==============================] - 98s 39ms/step - loss: 0.6577 - accuracy: 0.8175 - ignore_accuracy: 0.1322 - val_loss: 0.6516 - val_accuracy: 0.8185 - val_ignore_accuracy: 0.1334
Epoch 4/40
2504/2504 [==============================] - 98s 39ms/step - loss: 0.6322 - accuracy: 0.8206 - ignore_accuracy: 0.1358 - val_loss: 0.6317 - val_accuracy: 0.8194 - val_ignore_accuracy: 0.1356
Epoch 5/40
2504/2504 [==============================] - 97s 39ms/step - loss: 0.6133 - accuracy: 0.8219 - ignore_accuracy: 0.1393 - val_loss: 0.6

In [19]:
model.save('pos_tagger.h5')

In [20]:
scores = model.evaluate(test_sentences_X, to_categorical(test_tags_y, len(tag2index)))
print(f"{model.metrics_names[1]}: {scores[1] * 100}")

783/783 [==============================] - 17s 22ms/step
accuracy: 98.20792078971863


In [31]:
test_samples = [
    "I should speak to the principal about this incident .".split()
]
print(test_samples)

[['I', 'should', 'speak', 'to', 'the', 'principal', 'about', 'this', 'incident', '.']]


In [32]:
test_samples_X = []
for s in test_samples:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])
    test_samples_X.append(s_int)
 
test_samples_X = pad_sequences(test_samples_X, maxlen=MAX_LENGTH, padding='post')
print(test_samples_X)

[[5049 8812 1104 3890 8001 7914 3954 7716 9408 6912    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0]]


In [33]:
predictions = model.predict(test_samples_X)
print(predictions, predictions.shape)

[[[2.19085734e-07 7.02968426e-03 3.17033427e-03 ... 4.63619401e-08
   1.45397706e-07 1.05748586e-04]
  [5.56370416e-09 1.89385731e-02 1.20124989e-03 ... 5.73293235e-08
   3.14482918e-06 1.98203761e-05]
  [1.20179648e-05 4.01033030e-05 1.57566537e-04 ... 6.57654891e-05
   1.17047566e-05 7.34910063e-05]
  ...
  [9.99986172e-01 8.48272262e-17 9.35486688e-11 ... 5.46038095e-08
   4.01132354e-08 4.48325460e-10]
  [9.99979496e-01 1.46127967e-16 1.48098159e-10 ... 5.14599172e-08
   5.72895544e-08 9.13636278e-10]
  [9.99968529e-01 2.46559671e-16 2.32060746e-10 ... 4.94552346e-08
   7.64795161e-08 1.71940306e-09]]] (1, 125, 47)


In [34]:
def logits_to_tokens(sequences, index):
    token_sequences = []
    for categorical_sequence in sequences:
        token_sequence = []
        for categorical in categorical_sequence:
            token_sequence.append(index[np.argmax(categorical)])
 
        token_sequences.append(token_sequence)
 
    return token_sequences

In [35]:
predictions = model.predict(test_samples_X)
print(logits_to_tokens(predictions, {i: t for t, i in tag2index.items()}))

[['PRP', 'MD', 'VB', 'TO', 'DT', 'NN', 'IN', 'DT', 'NN', '.', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-